In [152]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import Chrome, ChromeOptions
from selenium.common.exceptions import NoSuchElementException
import time
import random
from tqdm import notebook
import pandas as pd
from datetime import datetime

In [153]:
# 所有需抓取的shop_id列表
data = pd.read_csv('shopid.csv')
shop_url_list = data['id'].unique()
shop_url = shop_url_list[0]

In [154]:
# 店铺url
basic_url = 'http://www.dianping.com/shop/'

In [155]:
yz_err_list = [] # 因为验证未通过无法抓取的id
page_err_list = [] # 因为网页结构问题无法抓取的id

In [ ]:
scraped = []
for i in [csv for csv in os.listdir() if csv.find('tempsave') != -1]: # 所有已被抓取的都被存入以tempsave开头的csv文件中
    scraped += list(pd.read_csv(i)['id'].unique())
scraped_set = set(scraped)

In [ ]:
# '''
# Input: 
# remain_set: ids to be scraped
# proxy_num: the port that mitmdump is listening to 
# webdriver_path: where your webdriver launcher is 
# '''

remain_set = set(shop_url_list) - scraped_set - set(yz_err_list) - set(page_err_list)
print(len(remain_set)) # 需被抓取的url的数量
proxy_num = '8080'
webdriver_path = '/webdriver_path'


In [316]:

scraped_list = []
err_list = []

# set Chromedriver options to conceal webdriver feature
option = ChromeOptions()
option.add_experimental_option('useAutomationExtension', False)
option.add_experimental_option('excludeSwitches', ['enable-automation'])
option.add_argument("--disable-blink-features")
option.add_argument("--disable-blink-features=AutomationControlled")
PROXY = f"127.0.0.1:{proxy_num}"
option.add_argument('--proxy-server=%s'%PROXY)
driver = webdriver.Chrome(webdriver_path
                        , options=option)


for shop_url_idx in notebook.tqdm(range(len(remain_set))):
    try:
        # save every 150 ids
        if (shop_url_idx%150 == 0) & (shop_url_idx != 0):
            temp_save = pd.DataFrame(scraped_list, columns = ['id', 'comment_text'])
            temp_save.to_csv(f'tempsave_{str(datetime.now())}.csv', index = False)
            print(f'saved to tempsave_{str(datetime.now())}.csv')
            time.sleep(50)
        
        url1 = basic_url + list(remain_set)[shop_url_idx]        
        contents = driver.get(url1)
        time.sleep(random.randint(8, 12))
        
        # id == summaryfilter-wrapper, for comments
        res = driver.find_element_by_id('summaryfilter-wrapper').text
        scraped_list.append((list(remain_set)[shop_url_idx], res))

    except NoSuchElementException:
        print(url1)
        # 每次保存都存一下以防万一
        temp_save = pd.DataFrame(scraped_list, columns = ['id', 'comment_text'])
        temp_save.to_csv(f'tempsave{str(datetime.now())}.csv', index = False)
        while True: 
            if "验证中心" in driver.page_source:
                yz_err_list.append(list(remain_set)[shop_url_idx])
                time.sleep(10)
            else:
                break
        if "验证中心" not in driver.page_source:
            try:
                res = driver.find_element_by_id('summaryfilter-wrapper').text
                scraped_list.append((list(remain_set)[shop_url_idx], res))
                time.sleep(8)
            except NoSuchElementException:
                page_err_list.append(list(remain_set)[shop_url_idx])
                time.sleep(4)
        pass



69


/home/willa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:20: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


  0%|          | 0/69 [00:00<?, ?it/s]

/home/willa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:35: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


http://www.dianping.com/shop/l5KcOyEmgIjHag9c
http://www.dianping.com/shop/k9r4bP6nCWNOHKX2
http://www.dianping.com/shop/G2lEQPk0D7SBro3B


In [314]:
temp_save = pd.DataFrame(scraped_list4, columns = ['id', 'comment_text'])
temp_save.to_csv(f'tempsave{str(datetime.now())}.csv', index = False)

# header

In [31]:
import requests
from bs4 import BeautifulSoup

In [ ]:
def webcapture(url):
    headers = {
   # pretend I am a browser
   'User-Agent': 'Mozilla/5.0',
   }
    session = requests.Session() #setup session
    data = session.get(url, headers=headers) #scrape the data
    soup = BeautifulSoup(data.text, 'html.parser') #parse the data
    return soup #return the parsed data


In [32]:
def craft_dp_comment_header(urlid):
    headers = {
        'Cookie': 's_ViewType=10; _lxsdk_cuid=1753fe5ef134c-0c735d38376786-c781f38-e1000-1753fe5ef14c8; _lxsdk=1753fe5ef134c-0c735d38376786-c781f38-e1000-1753fe5ef14c8; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; _hc.v=23325fa6-3ee9-a6a1-f004-fd67d15be960.1636200883; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1636200883; fspop=test; cy=1; cye=shanghai; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1636202836; _lxsdk_s=17cf52c20c6-50e-a96-ae9%7C%7C113',
        'Host': 'www.dianping.com',
        'Proxy-Connection': 'keep-alive',
        'Referer': f'http://www.dianping.com/shop/{urlid}',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
    }
    session = requests.Session() #setup session
    data = session.get(url, headers=headers) #scrape the data
    soup = BeautifulSoup(data.text, 'html.parser') #parse the data
    return soup #return the parsed data

In [106]:
url = 'http://www.dianping.com/shop/iNEqdrWJQZ0G69Ts'

In [107]:
headers = {
        'Cookie': 's_ViewType=10; _lxsdk_cuid=1753fe5ef134c-0c735d38376786-c781f38-e1000-1753fe5ef14c8; _lxsdk=1753fe5ef134c-0c735d38376786-c781f38-e1000-1753fe5ef14c8; _lx_utm=utm_source%3Dgoogle%26utm_medium%3Dorganic; _hc.v=23325fa6-3ee9-a6a1-f004-fd67d15be960.1636200883; Hm_lvt_602b80cf8079ae6591966cc70a3940e7=1636200883; fspop=test; cy=1; cye=shanghai; Hm_lpvt_602b80cf8079ae6591966cc70a3940e7=1636202836; _lxsdk_s=17cf52c20c6-50e-a96-ae9%7C%7C113',
        'Host': 'www.dianping.com',
        'Proxy-Connection': 'keep-alive',
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36'
    }
session = requests.Session() #setup session
data = session.get(url, headers=headers) #scrape the data

In [108]:
data


<Response [200]>

In [35]:
comment_url = 'http://www.dianping.com/ajax/json/shopDynamic/allReview?shopId=H51KYTdbxIDejKup&cityId=13&shopType=10&tcv=mn7fl73gmx&_token=eJxVj1tvgkAQhf%2FLvkpg2UVuSR802BYUUmHFSuMDN5FScGVBtE3%2Fe5fYPjSZ5Jz5Zk4y8wVaOwOmDCFUZAFc8haYQBahqAIBdIxPVKwiiBA2kI4EkP5nBuahpA0tYL7JSIfCVNP2I%2FE5uBMNwb1wt6rOLVJ4jTs2XwHHrqOmJA3DIGZl3NCyKcT0VEvseKLS81Re7kiWXG0rf1%2F2lJ8EeLImY1IzdAFjPIJqBFzjX%2B3%2Bepc%2Fw0OsLBrucudKAqaw88F3GdnAkLFLUFUdK7EXwA%2Fvpl9XJFy0n6y4zFyq1HOq%2BkpcOY%2Bh0p%2F9Oo%2FSbeGG1rLogmQNPdmmqxS7pLy1xqLxSPUabB3nEE37OKKT7e5JPTRhWcxnLxPD1jcP4PsHSUZpnA%3D%3D&uuid=23325fa6-3ee9-a6a1-f004-fd67d15be960.1636200883&platform=1&partner=150&optimusCode=10&originUrl=http%3A%2F%2Fwww.dianping.com%2Fshop%2FH51KYTdbxIDejKup'

# 尝试
https://www.jianshu.com/p/8326186c0273

In [330]:
process_cookie(refresh_cookie = True)

FileNotFoundError: [Errno 2] No such file or directory: 'cookie/03.cookies.txt'

In [331]:

import random
import requests
import json
from selenium import webdriver
from time import sleep
from bs4 import BeautifulSoup as BS


def login():
    """
    登录函数
    """
    login_url = 'https://account.dianping.com/login?redir=http%3A%2F%2Fwww.dianping.com%2F'
    driver.get(login_url)   # 打开登录页面，手动扫码登录

    while True:
        try:
            driver.find_element_by_class_name('login-container')  # 还能找到"请登录"
            sleep(2)
        except:
            driver.find_element_by_class_name('countDown')  # 登录成功
            break

    print('登录成功')


def process_cookie(refresh_cookie=True):
    """
    处理cookie，将driver 的 cookie复制到 requests，并把cookie保存下来供下次使用
    :param refresh_cookie: 是否从selenium更新cookie
    :return:
    """
    headers = {
        'User-Agent': UA,
        'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9',
        'Accept-Language': 'zh-CN,zh;q=0.9,en-US;q=0.8,en;q=0.7',
        'Upgrade-Insecure-Requests': '1',
        'Cache-Control': 'max-age=0',
    }
    session.headers.update(headers)
    # 将selenium中的cookie复制到requests.session()中
    if refresh_cookie:
        cookie = {cookie['name']: cookie['value'] for cookie in driver.get_cookies()}
        # 将获取到的cookie保存下来，避免重复获取
        with open('03.cookies.txt', 'w') as f:
            f.write(json.dumps(driver.get_cookies()))
        driver.quit()
        print('从selenium更新cookie')
    else:
        with open('03.cookies.txt', 'r') as f:
            cookies = json.loads(f.read())
            cookie = {_cookie['name']: _cookie['value'] for _cookie in cookies}
        print('从本地缓存更新cookie')
    session.cookies.update(cookie)


def fetch_shoplist(page_num=1):
    """
    获取美食分区下的店铺信息
    """
    city = 'guangzhou'
    food_shop_url = f'http://www.dianping.com/{city}/ch10'  # /p1~
    shop_info = []
    for i in range(1, page_num + 1):
        # driver.get(food_shop_url + f'/p{i}')
        res = session.get(food_shop_url + f'/p{i}')
        # soup = BS(driver.page_source, 'lxml')
        soup = BS(res.content, 'lxml')
        page_shop = soup.find_all('div', class_='txt')
        for shop in page_shop:
            shop_info.append({
                'shopid': shop.a['data-shopid'],
                'name': shop.a.text.strip(),
                'score': shop.find('div', class_='nebula_star').text.strip()
            })
    return shop_info


def fetch_comment(shopid='H7fXoNAkafmesfFG', page=1):
    """
    爬取指定店铺下的用户评论
    :param shopid: 店铺ID
    :param page: 页码
    :return: 解析后的评论信息
    """
    url = f'http://www.dianping.com/shop/{shopid}/review_all'
    if page != 1:
        url += f'/p{page}'
    # driver.get(url)
    # soup = BS(driver.page_source, 'lxml')
    res = session.get(url)
    soup = BS(res.text, 'lxml')
    comments = soup.find_all('div', class_='main-review')
    comment_list = []
    for item in comments:   # 遍历所有评论
        username = item.find('div', class_='dper-info').text.strip()
        items = item.find_all('span', class_='item')     # 各项评分
        detail_score = []
        for _item in items:
            detail_score.append(_item.text.strip())
        content = item.find('div', class_='review-words').text.strip()  # 获取到的评论不全，做了CSS加密
        comment_list.append({'username': username, 'item': detail_score, 'content': content})
    return comment_list




In [327]:

refresh_cookie = True
UA = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.128 Safari/537.36'

session = requests.session()
session.headers.clear()

if refresh_cookie:
    options = webdriver.ChromeOptions()
    options.add_argument(f'user-agent={UA}')

    options.add_experimental_option("excludeSwitches", ["enable-automation"])
    options.add_experimental_option('useAutomationExtension', False)

    # 没有配置环境变量的话需要填写Chromedriver的路径：webdriver.Chrome(executable_path="***")
    driver = webdriver.Chrome('/mnt/c/Users/Willa/Downloads/chromedriver_win32/chromedriver.exe'
                              , options=options)
    driver.maximize_window()

    # 去掉window.navigator.webdriver字段，防止被检测出是使用selenium
    driver.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
        "source": """
        Object.defineProperty(navigator, 'webdriver', {
          get: () => undefined
        })
      """
    })

    login()
process_cookie(refresh_cookie)
shop_info = fetch_shoplist()
for _ in shop_info:
    print(_)

shopid = shop_info[2]['shopid']  # 这里也可以用for来遍历，但要适当加延时
comment_list = fetch_comment(shopid=shopid, page=1)
# print(comment_list)
for _ in comment_list:
    print(_)

/home/willa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  app.launch_new_instance()
/home/willa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:18: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
/home/willa/.local/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead


NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".countDown"}
  (Session info: chrome=95.0.4638.69)
Stacktrace:
Backtrace:
	Ordinal0 [0x00BF06F3+2492147]
	Ordinal0 [0x00B89BD1+2071505]
	Ordinal0 [0x00A92478+1057912]
	Ordinal0 [0x00ABC964+1231204]
	Ordinal0 [0x00AE6B62+1403746]
	Ordinal0 [0x00AD57FA+1333242]
	Ordinal0 [0x00AE4F38+1396536]
	Ordinal0 [0x00AD568B+1332875]
	Ordinal0 [0x00AB21D4+1188308]
	Ordinal0 [0x00AB302F+1191983]
	GetHandleVerifier [0x00D767A6+1545030]
	GetHandleVerifier [0x00E2105C+2243580]
	GetHandleVerifier [0x00C7BC97+518199]
	GetHandleVerifier [0x00C7AD80+514336]
	Ordinal0 [0x00B8ED2D+2092333]
	Ordinal0 [0x00B92EE8+2109160]
	Ordinal0 [0x00B93022+2109474]
	Ordinal0 [0x00B9CB71+2149233]
	BaseThreadInitThunk [0x7638FA29+25]
	RtlGetAppContainerNamedObjectPath [0x77AD7A9E+286]
	RtlGetAppContainerNamedObjectPath [0x77AD7A6E+238]


In [333]:
session.get('http://www.dianping.com/shop/l85OGfGaDg6txGPV').text

'<!DOCTYPE html> <html> <head> <title>【敏华冰厅(天河城店)】电话,地址,价格,营业时间(图) - 广州美食 - 大众点评网</title> <link rel="icon" type="image/x-icon" href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico"> <link rel="shortcut icon" type="image/x-icon" href="//www.dpfile.com/app/pc-common/dp_favicon.a4af753914321c8e82e402e2b4be01d7.ico" > <link rel="apple-touch-icon" href="//www.dpfile.com/s/res/app-touch-icon.89213f53ed66e1693e4b6aeedd355349.png"> <link rel="apple-touch-icon" sizes="76x76" href="//www.dpfile.com/s/res/app-touch-icon-76x76.6399ce382f3e0584a6b07599cbeb3bcb.png"> <link rel="apple-touch-icon" sizes="120x120" href="//www.dpfile.com/s/res/app-touch-icon-120x120.067844b8518f076b154dcf793a46a0a5.png"> <link rel="apple-touch-icon" sizes="152x152" href="//www.dpfile.com/s/res/app-touch-icon-152x152.ee6d0c24fc2de0f9a62b6cc9e6720393.png"> <link rel="canonical" href="http://www.dianping.com/shop/l85OGfGaDg6txGPV"/> <link rel="alternate" media="only screen and (max-width: 64

In [332]:
process_cookie(refresh_cookie)
shop_info = fetch_shoplist()
for _ in shop_info:
    print(_)

shopid = shop_info[2]['shopid']  # 这里也可以用for来遍历，但要适当加延时
comment_list = fetch_comment(shopid=shopid, page=1)
# print(comment_list)
for _ in comment_list:
    print(_)

从selenium更新cookie
{'shopid': 'G3w49SwIgBbzaiBX', 'name': '粤集·粤菜正牌(长隆店)', 'score': ''}
{'shopid': 'H7fXoNAkafmesfFG', 'name': '惠食佳(东风店)', 'score': ''}
{'shopid': 'Gams6NXcwGShysQe', 'name': '惠食佳(滨江大公馆店)', 'score': ''}
{'shopid': 'G8COFTq4XL5CzptG', 'name': '永兴烧腊(黄沙店)', 'score': ''}
{'shopid': 'k6pvmplP2qbk4aDx', 'name': '荔園 LaiYuen', 'score': ''}
{'shopid': 'k42yR5E22kRaBpsk', 'name': 'COLINMINT简悦厨(富力海珠城店)', 'score': ''}
{'shopid': 'k8uScYQNiUgBiisz', 'name': '滋粥楼顺德菜(南村店)', 'score': ''}
{'shopid': 'k2q4US1CMl9qwksR', 'name': '居心地屋かくら弐代目', 'score': ''}
{'shopid': 'G2Rkv9xU6nLUDuIO', 'name': '尚莲·越泰LIAN Viet·Thai Cuisine(IGC天汇店)', 'score': ''}
{'shopid': 'H1J6jh7ZQJTRaqEI', 'name': 'BE BRIGHT KITCHEN(正佳广场店)', 'score': ''}
{'shopid': 'k4eU9aCSJl7fTjWp', 'name': '摩打食堂(珠影星光城店)', 'score': ''}
{'shopid': 'l85OGfGaDg6txGPV', 'name': '敏华冰厅(天河城店)', 'score': ''}
{'shopid': 'G3KjI2zA0Hoo2IZF', 'name': '禄运茶居(体育西店)', 'score': ''}
{'shopid': 'ESAOBxzeXoQ1SkRs', 'name': '和苑酒家(时代店)', 'score': ''}
{'shopi